# APIM ❤️ OpenAI

## GPT-4o inferencing lab
![flow](../../images/GPT-4o-inferencing.gif)

Playground to try the new GPT-4o model. GPT-4o ("o" for "omni") is designed to handle a combination of text, audio, and video inputs, and can generate outputs in text, audio, and image formats.

### TOC
- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Create the Azure Resource Group](#1)
- [2️⃣ Create deployment using 🦾 Bicep](#2)
- [3️⃣ Get the deployment outputs](#3)
- [🧪 Test the API using the Azure OpenAI Python SDK](#sdk1)
- [🧪 Image Processing](#sdk2)
- [🧪 Base64 Image Processing](#sdk3)
- [🗑️ Clean up resources](#clean)

### Prerequisites
- [Python 3.8 or later version](https://www.python.org/) installed
- [Pandas Library](https://pandas.pydata.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/free/) with Contributor permissions
- [Access granted to Azure OpenAI](https://aka.ms/oai/access) or just enable the mock service
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id
- The ```mock_webapps``` variable sets the list of deployed Web Apps for the mocking functionality. Clean the ```openai_resources``` list to simulate the OpenAI behaviour with the mocking service.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [3]:
import os

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "westeurope"

openai_resources = [
    {"name": "openai1", "location": "westus3"}
]
openai_model_name = "gpt-4o"
openai_model_version = "2024-05-13"
openai_deployment_name = "gpt-4o"
openai_api_version = "2024-02-01"


<a id='1'></a>
### 1️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [1]:
# %load ../../shared/snippets/create-az-resource-group.py

# type: ignore

import datetime

resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location}

if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print(f"✅ Azure Resource Group {resource_group_name} created ⌚ {datetime.datetime.now().time()}")


["ERROR: (InvalidResourceGroup) The provided resource group name '{resource_group_name}' has these invalid characters: '{}'. The name can only be a letter, digit, '-', '.', '(', ')' or '_'. For more details, visit https://aka.ms/ResourceGroupNamingRestrictions .", 'Code: InvalidResourceGroup', "Message: The provided resource group name '{resource_group_name}' has these invalid characters: '{}'. The name can only be a letter, digit, '-', '.', '(', ')' or '_'. For more details, visit https://aka.ms/ResourceGroupNamingRestrictions ."]


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [5]:
# %load ../../shared/snippets/create-az-deployment.py

# type: ignore

import json

backend_id = "openai-backend-pool" if len(openai_resources) > 1 else openai_resources[0].get("name")

with open("policy.xml", 'r') as policy_xml_file:
    policy_xml = policy_xml_file.read()

    if backend_id:
        policy_xml = policy_xml.replace("{backend-id}", backend_id)

    if client_id:
        policy_xml = policy_xml.replace("{aad-client-application-id}", client_id)

    if tenant_id:
        policy_xml = policy_xml.replace("{aad-tenant-id}", tenant_id)

    policy_xml_file.close()
open("policy-updated.xml", 'w').write(policy_xml)

bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "openAIConfig": { "value": openai_resources },
        "openAIDeploymentName": { "value": openai_deployment_name },
        "openAIModelName": { "value": openai_model_name },
        "openAIModelVersion": { "value": openai_model_version },
        "openAIAPIVersion": { "value": openai_api_version }
    }
}

with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

! az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file "main.bicep" --parameters "params.json"


<a id='3'></a>
### 3️⃣ Get the deployment outputs

We are now at the stage where we only need to retrieve the gateway URL and the subscription before we are ready for testing.

In [8]:
# %load -r 1-21 ../../shared/snippets/deployment-outputs.py
# type: ignore

stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimServiceId.value -o tsv
apim_service_id = stdout.n
print(f"👉🏻 APIM Service Id: {apim_service_id}")

stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimSubscriptionKey.value -o tsv
apim_subscription_key = stdout.n
print(f"👉🏻 APIM Subscription Key (masked): ****{apim_subscription_key[-4:]}")

stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimResourceGatewayURL.value -o tsv
apim_resource_gateway_url = stdout.n
print(f"👉🏻 APIM API Gateway URL: {apim_resource_gateway_url}")

stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.logAnalyticsWorkspaceId.value -o tsv
workspace_id = stdout.n
print(f"👉🏻 Workspace ID: {workspace_id}")

stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.applicationInsightsAppId.value -o tsv
app_id = stdout.n
print(f"👉🏻 App ID: {app_id}")

<a id='requsts'></a>
### 🧪 Test the API using a direct HTTP call
Requests is an elegant and simple HTTP library for Python that will be used here to make raw API requests and inspect the responses.

In [ ]:
import requests
import time

runs = 1
sleep_time_ms = 1000
url = apim_resource_gateway_url + "/openai/deployments/" + openai_deployment_name + "/chat/completions?api-version=" + openai_api_version

for i in range(runs):
    print(f"▶️ Run {i+1}/{runs}:")

    messages = {"messages": [
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]}

    response = requests.post(url, headers = {'api-key': apim_subscription_key}, json = messages)

    print("status code: ", response.status_code)
    print("headers ", response.headers)
    print("x-ms-region: ", response.headers.get("x-ms-region")) # this header is useful to determine the region of the backend that served the request

    if (response.status_code == 200):
        data = json.loads(response.text)
        print("response: ", data.get("choices")[0].get("message").get("content"))
    else:
        print(response.text)

    time.sleep(sleep_time_ms/1000)


<a id='sdk1'></a>
### 🧪 Test the API using the Azure OpenAI Python SDK


In [ ]:
import time
from openai import AzureOpenAI

runs = 1
sleep_time_ms = 1000

client = AzureOpenAI(
    azure_endpoint = apim_resource_gateway_url,
    api_key = apim_subscription_key,
    api_version = openai_api_version
)

for i in range(runs):
    print(f"▶️ Run {i+1}/{runs}:")

    messages = [
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]

    response = client.chat.completions.create(model = openai_model_name, messages = messages)
    print("💬 ", response.choices[0].message.content)
    time.sleep(sleep_time_ms/1000)


<a id='sdk2'></a>
### 🧪 Image Processing

GPT-4o can directly process images and take intelligent actions based on the image. 

The following example is adapted from the [OpenAI Cookbook](https://cookbook.openai.com/examples/gpt4o/introduction_to_gpt4o)

#### Input image:
![image](https://upload.wikimedia.org/wikipedia/commons/e/e2/The_Algebra_of_Mohammed_Ben_Musa_-_page_82b.png)





In [ ]:
from IPython.display import display, Markdown

response = client.chat.completions.create(
    model = openai_model_name,
    messages = [
        {"role": "system", "content": "You are a helpful assistant that responds in Markdown. Help me with my math homework!"},
        {"role": "user", "content": [
            {"type": "text", "text": "What's the area of the triangle?"},
            {"type": "image_url", "image_url": {
                "url": "https://upload.wikimedia.org/wikipedia/commons/e/e2/The_Algebra_of_Mohammed_Ben_Musa_-_page_82b.png"}
            }
        ]}
    ],
    temperature = 0.0,
)

display(Markdown(response.choices[0].message.content))


<a id='sdk3'></a>
### 🧪 Base64 Image Processing

#### Input image:
<img src="meme.png" width="600px">


In [ ]:
from IPython.display import display, Markdown, HTML
import base64

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

base64_image = encode_image("meme.png")

response = client.chat.completions.create(
    model = openai_model_name,
    messages = [
        {"role": "system", "content": "You are a helpful assistant that helps to interprete memes!"},
        {"role": "user", "content": [
            {"type": "text", "text": "What's funny about this picture?"},
            {"type": "image_url", "image_url": {
                "url": f"data:image/png;base64,{base64_image}"}
            }
        ]}
    ],
    temperature = 0.0,
)

HTML('<font size="5">' + response.choices[0].message.content + '</font>')

<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.